In [1]:
import sys
sys.path.insert(0,'python')
from fastRWpkl import *
from scipy import signal
import numpy as np
import scipy
from scipy import signal
from scipy.ndimage.morphology import binary_dilation as bd
import scipy.ndimage as ndimage
from functools import partial
import multiprocessing
import numpy.ma as ma
from collections import Counter
import cPickle as pkl
import pylab as plt
%matplotlib inline

In [2]:
inds = pkl.load(open('pkls/gaus_trans.pkl','rb'))
data = parallel_rw_pkl(None, 'test_patch', 'r')
mask = parallel_rw_pkl(None, 'mtest_patch', 'r')
modis_sent = ma.array(data, mask=mask)

Sent = parallel_rw_pkl(None, 'b4test_patch', 'r')
stm = parallel_rw_pkl(None, 'stmtest_patch', 'r')
p_mask = np.unique(stm)

struct = ndimage.generate_binary_structure(2, 2)

In [39]:
def gaussian(xwin, ywin, xstd, ystd, angle, norm = True):
    win = max(xwin, ywin)
    winx = win*2**0.5
    winy = win*2**0.5
    
    xstd = xstd*2**0.5
    ystd = ystd*2**0.5
        
    xgaus = signal.gaussian(winx, xstd)
    ygaus = signal.gaussian(winy, ystd)
    gaus  = np.outer(xgaus, ygaus)
    r_gaus = scipy.ndimage.interpolation.rotate(gaus, angle, reshape=True)
    center = np.array(r_gaus.shape)/2
    cgaus = r_gaus[center[0]-xwin/2.: center[0]+xwin/2., center[1]-ywin/2.:center[1]+ywin/2.]
    if norm:
        return cgaus/cgaus.sum()
    else:
        return cgaus


def conv(p, Sent, modis_sent, inds,c_pixels):
    
    xstd,ystd,angle, sx, sy = p
    print p
    
    xwin,ywin = 100,100
    gaus = gaussian(xwin,ywin,xstd,ystd,angle,False)                            
    '''
    kernel = np.zeros((99, 225))
    kernel[inds[1], inds[0]] = gaus.ravel()

    if kernel.sum() <= 0:
        print 'invalid gauss: %s'%([xstd, ystd, angle])
        pass
    else:
        ker = kernel/(kernel.sum())
    '''
    ker = gaus/(gaus.sum())
    shape = 3000
    dia = 1000
    hdia = dia/2
    # for symetric dialation of the patche to avoid the border effects
    '''
    to_conv = np.zeros(np.array(Sent.shape)+dia)
    to_conv[:hdia,:hdia] = Sent[:hdia,:hdia][::-1,::-1]
    to_conv[:hdia,hdia:shape+hdia] = Sent[:hdia,::][::, ::-1]
    to_conv[hdia:shape+hdia, :hdia] = Sent[::,:hdia][::,::-1]
    to_conv[shape+hdia:, :hdia] = Sent[-hdia:,:hdia][::-1,::-1]
    to_conv[shape+hdia:,hdia:shape+hdia] = Sent[-hdia:, ::][::-1,::]
    to_conv[shape+hdia:,shape+hdia:] = Sent[-hdia:, -hdia:][::-1,::-1]
    to_conv[hdia:shape+hdia, shape+hdia:] = Sent[::, -hdia:][::,::-1]
    to_conv[:hdia,shape+hdia:] = Sent[:hdia,-hdia:][::-1,::-1]
    to_conv[hdia:shape+hdia, hdia:shape+hdia] = Sent
    '''
    
    to_conv = np.zeros(np.array(Sent.shape)*2)
    to_conv[:3000,:3000] = Sent[::-1,::-1]
    to_conv[3000:,:3000] = Sent[::,::-1]
    to_conv[3000:,3000:] = Sent
    to_conv[:3000,3000:] = Sent[::-1,::]
    
    shx = hdia+sx; shy = hdia + sy
    s = signal.fftconvolve(to_conv, ker, mode='valid')[3000:,3000:]
    shape = s.shape

    #s = signal.fftconvolve(to_conv, ker, mode='valid')[shx:shape+shx,shy:shape+shy]
    m = (modis_sent*0.001)[:shape[0], :shape[1]]
    ma = (c_pixels[0] >= shape[0]) | (c_pixels[1] >= shape[1])
    c_pixels = np.array([c_pixels[0][~ma], c_pixels[1][~ma]])
    
    vm = m[c_pixels[0], c_pixels[1]]
    vs = s[c_pixels[0], c_pixels[1]]
    vm = vm[~vm.mask]; vs = vs[~vm.mask]    
    r = scipy.stats.linregress(vm,vs)
    cost = 1-r.rvalue
    print cost
    return cost

In [ ]:
from scipy import optimize
c_pixels = np.array(parallel_rw_pkl(None, 'c_pixels', 'r')).T
p = np.array([82.,142., 121., 3, -7])
bound = np.array([(10.,144.),(75.,205.),(62.,187.),(-50.,50.),(-50.,50.)])
psolve = optimize.fmin_l_bfgs_b(conv,p,approx_grad=True,iprint=-1,\
                                args=(Sent, modis_sent, inds,c_pixels),bounds=bound)